### GOAL: Combine columns in dataset cleanly using sql to avoid indexing errors
#### Scott Masterson, Applied Math and Statistics 
###### Emory University

In [149]:
import pandas as pd
import numpy as np

#sql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

In [160]:
#read data set from excel file on my computer
df = pd.read_excel('/Users/scottmasterson/Documents/Resume_Job/All_Course_Offerings.xlsx', 
                   usecols = 'A:T', skiprows = 5)

df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
df.columns.name = None


#Check columns to view unexpected inputs
list(df.columns.values)

['Term',
 'Subject',
 'Catalogs',
 'Sections',
 'Programs',
 nan,
 'Title',
 'Max Units',
 'Instructor',
 'Bldg & Rm',
 nan,
 'M',
 'TU',
 'W',
 'TH',
 'F',
 'Start ',
 'End',
 'Capacity',
 'Enr']

In [161]:
#rename and check to avoid errors
df = df.rename(columns={"Start ": "Start"})
df.head()

,Term,Subject,Catalogs,Sections,Programs,NaN,Title,Max Units,Instructor,Bldg & Rm,NaN,M,TU,W,TH,F,Start,End,Capacity,Enr
0,5229,ACT,710,5102,PHD,NaN,Accounting Research I,3,"Dichev,Ilia D.",GFC_W520,NaN,M,NaN,NaN,NaN,NaN,09:00,12:00,66,3
1,5229,ACT,200,1101,BBA,NaN,Accounting:The Language of Bus,3,"Rackliffe,Usha",GBS_130,NaN,M,NaN,W,NaN,NaN,13:00,14:15,62,6
2,5229,ACT,200,1102,BBA,NaN,Accounting:The Language of Bus,3,"Rackliffe,Usha",GBS_130,NaN,M,NaN,W,NaN,NaN,14:30,15:45,62,9
3,5229,ACT,200,1103,BBA,NaN,Accounting:The Language of Bus,3,"Zhao,Jianxin",GBS_304,NaN,NaN,TU,NaN,TH,NaN,10:00,11:15,62,58
4,5229,ACT,200,1104,BBA,NaN,Accounting:The Language of Bus,3,"Zhao,Jianxin",GBS_130,NaN,NaN,TU,NaN,TH,NaN,13:00,14:15,62,62


In [162]:
#select inportant columns
df_trim = df[['Subject', 'Instructor', 'M', 'TU', 'W', 'TH', 'F', 'Start', 'End']]

#create new columns
df_trim['Time'] = df_trim['Start'] + ' ' + '-' + ' ' + df_trim['End']
df_trim.head()

<ipython-input-162-a51d2287d9d1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim['Time'] = df_trim['Start'] + ' ' + '-' + ' ' + df_trim['End']


,Subject,Instructor,M,TU,W,TH,F,Start,End,Time
0,ACT,"Dichev,Ilia D.",M,NaN,NaN,NaN,NaN,09:00,12:00,09:00 - 12:00
1,ACT,"Rackliffe,Usha",M,NaN,W,NaN,NaN,13:00,14:15,13:00 - 14:15
2,ACT,"Rackliffe,Usha",M,NaN,W,NaN,NaN,14:30,15:45,14:30 - 15:45
3,ACT,"Zhao,Jianxin",NaN,TU,NaN,TH,NaN,10:00,11:15,10:00 - 11:15
4,ACT,"Zhao,Jianxin",NaN,TU,NaN,TH,NaN,13:00,14:15,13:00 - 14:15


In [165]:
#encountered an indexing error here that didn't allow the selection of 'Time'
df_error = df[['Time']]

KeyError: "None of [Index(['Time'], dtype='object')] are in the [columns]"

In [166]:
#use sql to bypass error
clean_data = mysql('''SELECT Subject, Instructor, M, TU, W, TH, F, Time
                      FROM df_trim''')

print(type(clean_data))

#check to see if sql resolved issue
clean_data[['Time']]

<class 'pandas.core.frame.DataFrame'>


,Time
0,09:00 - 12:00
1,13:00 - 14:15
2,14:30 - 15:45
3,10:00 - 11:15
4,13:00 - 14:15
...,...
267,None
268,None
269,None
270,None


In [167]:
#final dataset
clean_data

,Subject,Instructor,M,TU,W,TH,F,Time
0,ACT,"Dichev,Ilia D.",M,None,None,None,None,09:00 - 12:00
1,ACT,"Rackliffe,Usha",M,None,W,None,None,13:00 - 14:15
2,ACT,"Rackliffe,Usha",M,None,W,None,None,14:30 - 15:45
3,ACT,"Zhao,Jianxin",None,TU,None,TH,None,10:00 - 11:15
4,ACT,"Zhao,Jianxin",None,TU,None,TH,None,13:00 - 14:15
...,...,...,...,...,...,...,...,...
267,BUS,"Lord,Michael David",None,None,None,None,None,None
268,BUS,"Lord,Michael David",None,None,None,None,None,None
269,BUS,"Lord,Michael David",None,None,None,None,None,None
270,BUS,"Lord,Michael David",None,None,None,None,None,None
